In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

D:\anacondaz\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
D:\anacondaz\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
D:\anacondaz\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [6]:
data = pd.read_csv('retail_train.csv')
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [7]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

Сгруппируем данные по item_id и суммируем количество купленных товаров (возможно, что человек зашел в магазин 1 раз, но купил 10 пачек одного продукта)

In [42]:
data_g=data.groupby('item_id',as_index=False).aggregate({'quantity':'sum'}).rename(columns={"quantity":"popular"})

data_g['weights']=data_g['popular']/data_g.popular.sum()
data_g['weights']=data_g['weights'].round(7)
data_g.head()

,item_id,popular,weights
0,25671,6,0.0
1,26081,1,0.0
2,26093,1,0.0
3,26190,1,0.0
4,26355,2,0.0


In [34]:
import random

In [38]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    items = np.array(items_weights.item_id)
    weights=np.array(items_weights.weights)
    recs = random.choices(population=items, weights=weights,k=n) # добавил эту фукнцию чтобы учитывать веса
    
    
    return recs

In [36]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [39]:
%%time
# your_code
result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(data_g, n=5))
result.head(2)

Wall time: 2min 23s


,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6534178, 6534178, 6534178, 6534178, 6534178]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6534166, 6534178, 6534178, 6534178, 6534178]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [2]:
result = pd.read_csv('predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [108]:
def precision_at_k(recommended_list, bought_list, k=5):
#     парсим стринг в лист
    rec_l=list(str.split(recommended_list[1:-1],sep=', '))
    bou_l=[]
    for x in str.split(bought_list[1:-1],sep='\n'):
        c=str.rsplit(x,sep=' ')
        for n in c:
            if n!='':
                bou_l.append(n)
        
    bought_list = np.array(bou_l)
    recommended_list = np.array(rec_l)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[0:k]
    
    flags = np.isin(bought_list, recommended_list)

    precision = flags.sum() / len(recommended_list)

    return precision

In [109]:
# your_code

rec_lists=result.columns[2:]

for x in rec_lists:
    y=result.apply(lambda row:precision_at_k(row[x],row['actual']),axis=1).mean()
    print(f'precision_at_k для {x} : {y}')

precision_at_k для random_recommendation : 0.0005876591576885406
precision_at_k для popular_recommendation : 0.15523996082272082
precision_at_k для itemitem : 0.033594515181195064
precision_at_k для cosine : 0.03525954946131262
precision_at_k для tfidf : 0.03614103819784544
precision_at_k для own_purchases : 0.1799869409076044


### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [110]:
# your_code
# без звездочки